##Importação das bibliotecas

In [ ]:
!pip install pandera
!pip install gcsfs

In [ ]:
# lib para conexão
import requests

# Modelagem de dados
import pandas as pd
import pandera as pa

#Conexão com o GCP
from google.cloud import storage
import os

# Ignorando avisos
import warnings
warnings.filterwarnings('ignore')

##Conector GCP


In [ ]:
#CONFIGURAÇÃO DA CHAVE DE SEGURANÇA
serviceAccount = '/content/drive/MyDrive/Key/bc17-349312-c624582f6ec6.json' #Caminho da key para conexão com a GCP

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

##Requisição do dataset

In [ ]:
# URL da API da Alesp
URL_Despesas = 'http://www.al.sp.gov.br/repositorioDados/deputados/despesas_gabinetes.xml'

# Pegando os dados
conexao_01 = requests.get( URL_Despesas )

# Verificando se a API funcinou
print( conexao_01)

##Leitura do dataset

In [ ]:
dfDespesasOriginal = pd.read_xml(conexao_01.content)

In [ ]:
#Backup do DF original
dfDespesas = dfDespesasOriginal.copy() 

In [ ]:
dfDespesas

##Pré-ánalise

In [ ]:
dfDespesas.info()

In [ ]:
#Verificando inconsistências em cada coluna, realizar coluna por coluna
pd.unique(dfDespesas['Tipo'])

In [ ]:
#Verificando inconsistências em CNPJS válidos
cpfsValidos = dfDespesas.loc[dfDespesas.CNPJ.notnull()]
sorted(cpfsValidos["CNPJ"].unique(),reverse=True)

In [ ]:
#Verificando inconsistências na coluna fornecedor onde os CNPJs são inválidos
cnpjInvalidos = dfDespesas.loc[dfDespesas.CNPJ.isnull()]
cnpjInvalidos["Fornecedor"].unique()

Inconsistências encontradas na coluna Fornecedor onde os CNPJs são nulos:\
PEDAGIO = PEDÁGIO, PEDAGIO, PEDÁGIOS, CUPONS DE PEDÁGIO,PEDAGIOS, PEGAGIO, CONCESSIONARIA RODOVIA OESTE DE SAO PAULO S.A.\
TAXI = TAXI, DELTA RADIO TAXI

Inconsistências na coluna CNPJ onde os CNPJs são válidos\
['O7482309000104',
 'O4689799000126',
 'O4040553000129',
 'O2358317000120',
 'O1520430000105',]


In [ ]:
#Contando os valores nulos na coluna CNPJ
dfDespesas.CNPJ.isnull().sum()

Total de valores nulos na coluna CNPJ: 10725

##Tratamento dos dados

In [ ]:
#Mudando todos os valores semelhantes a PEDAGIO
dfDespesas["Fornecedor"].replace(["PEDÁGIO","PEDÁGIOS","CUPONS DE PEDÁGIO","PEGAGIO","PEDAGIOS","CONCESSIONARIA RODOVIA OESTE DE SAO PAULO S.A."],"PEDAGIO",inplace=True)

In [ ]:
#Mudando todos os valores semelhantes a TAXI
dfDespesas["Fornecedor"].replace("DELTA RADIO TAXI","TAXI", inplace=True)

In [ ]:
#Verificando as mudanças
cnpjInvalidos = dfDespesas.loc[dfDespesas.CNPJ.isnull()]
cnpjInvalidos["Fornecedor"].unique()

In [ ]:
#Retirando letras da coluna CNPJ
dfDespesas["CNPJ"].replace(r"[a-zA-Z]","",regex=True,inplace=True)

##Validação dos dados

In [ ]:
#Verificando valores nulos
dfDespesas.isna().sum()

In [ ]:
#Infos geral do DF
dfDespesas.info()

In [ ]:
from pandera.engines.numpy_engine import String
from pandera.schema_components import Column

#Construção do Schema para realizar a validação
schema = pa.DataFrameSchema(
    columns = {
        "Ano":pa.Column(pa.Int),
        "Matricula":pa.Column(pa.Int),
        "Mes":pa.Column(pa.Int),
        "Valor":pa.Column(pa.Float),
        "CNPJ":pa.Column(pa.String,nullable=True),
        "Deputado":pa.Column(pa.String),
        "Tipo":pa.Column(pa.String),
        "Fornecedor":pa.Column(pa.String)
    }
)

In [ ]:
#Validação
schema.validate(dfDespesas)

#Insights

##Insight 1
Valores total dos gastos por cada deputado nos ultimos 4 anos




In [ ]:
#Filtro dos 4 últimos naos
df4anos = dfDespesas.loc[dfDespesas['Ano']>2018]

In [ ]:
insight1 = df4anos.groupby(["Deputado"])["Valor"].sum().sort_values(ascending=False)

In [ ]:
insight1

##Insight 2
Mostrar os itens dos ultimos quatro anos onde se mais teve despesa 

In [ ]:
insight2 = df4anos.groupby(["Tipo"])["Valor"].sum().sort_values(ascending=False)

In [ ]:
insight2.plot.bar()

##Insight 3
Mostrar os 3 deputados que mais e menos gastam

In [ ]:
insight1.head(3).plot.bar(figsize=(5,5),ylabel="Valor",logy=True)

In [ ]:
insight1.tail(3).plot.bar(ylabel="Valor")

##Insight 4
5 fornecedores com mais gastos e menos gastos

In [ ]:
insight3 = dfDespesas.groupby("Fornecedor")["Valor"].sum().sort_values(ascending=False)
insight3.head(5)

In [ ]:
insight3.tail(5)

##Insight 5
Total de gastos em cada ano

In [ ]:
insight4 = dfDespesas.groupby("Ano")["Valor"].sum().sort_values(ascending=False)
insight4

#Load

In [ ]:
schema.validate(dfDespesas)

In [ ]:
#Carregamento do arquvio no formato CSV para a GCP 
dfDespesas.to_csv("gs://nome_bucket/Tratados/despesas_gabinetes.csv",index=False)